In [94]:
from pathlib import Path
import sqlalchemy as sa
import pandas as pd

db_path = Path("./eqr.db")
engine = sa.create_engine(f"sqlite:///{db_path}")

year = 2020
quarter = 'Q2'

## Get Contracts

In [105]:
with engine.connect() as conn:
    all_contracts = pd.read_sql_table("contracts", conn)

In [106]:
all_contracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778269 entries, 0 to 778268
Data columns (total 33 columns):
 #   Column                                 Non-Null Count   Dtype 
---  ------                                 --------------   ----- 
 0   contract_unique_id                     778269 non-null  object
 1   seller_company_name                    778269 non-null  object
 2   seller_history_name                    0 non-null       object
 3   customer_company_name                  778269 non-null  object
 4   contract_affiliate                     778269 non-null  object
 5   ferc_tariff_reference                  778241 non-null  object
 6   contract_service_agreement_id          778189 non-null  object
 7   contract_execution_date                778269 non-null  int64 
 8   commencement_date_of_contract_term     778269 non-null  int64 
 9   contract_termination_date              260131 non-null  object
 10  actual_termination_date                15911 non-null   object
 11  

In [108]:
contracts = all_contracts.query("year == @year & quarter == @quarter")

## Get Identities

In [112]:
with engine.connect() as conn:
    all_identities = pd.read_sql_table("identities", conn)

In [113]:
all_identities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22998 entries, 0 to 22997
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   filer_unique_id                                  22998 non-null  object
 1   company_name                                     22998 non-null  object
 2   company_identifier                               22998 non-null  object
 3   contact_name                                     22998 non-null  object
 4   contact_title                                    22998 non-null  object
 5   contact_address                                  22998 non-null  object
 6   contact_city                                     22998 non-null  object
 7   contact_state                                    22998 non-null  object
 8   contact_zip                                      22998 non-null  object
 9   contact_country_name                   

In [114]:
identities = all_identities.query("year == @year & quarter == @quarter")

## Does every `contracts.seller_company_name` exist in `identities.company_name`?

In [115]:
contract_company_names = pd.Series(contracts.seller_company_name.unique())
identity_company_names = pd.Series(identities.company_name.unique())
print(contract_company_names.shape)
print(identity_company_names.shape)

(2315,)
(2810,)


In [116]:
is_name_in_both = contract_company_names.isin(identity_company_names)
print(is_name_in_both.value_counts())

contract_company_names[~is_name_in_both]

True     2314
False       1
dtype: int64


1224    FirstLight Power Resources Management, LLC
dtype: object

In [117]:
contract_company_names[contract_company_names.str.contains("First")]

1143                  FirstLight CT Housatonic LLC
1144                       FirstLight CT Hydro LLC
1145                       FirstLight MA Hydro LLC
1222                       First Choice Energy LLC
1223               FirstLight Power Management LLC
1224    FirstLight Power Resources Management, LLC
2076                        First Point Power, LLC
dtype: object

In [118]:
identity_company_names[identity_company_names.str.contains("First")]

1449       FirstLight CT Housatonic LLC
1450            FirstLight CT Hydro LLC
1451            FirstLight MA Hydro LLC
1539            First Choice Energy LLC
1540    FirstLight Power Management LLC
2525             First Point Power, LLC
dtype: object

There is ONE seller company name in the contracts data that does not exist in the identity table. 

## All of 2020

In [122]:
contract_company_names = pd.Series(all_contracts.seller_company_name.unique())
identity_company_names = pd.Series(all_identities.company_name.unique())
print(contract_company_names.shape)
print(identity_company_names.shape)

(2483,)
(3005,)


In [123]:
is_name_in_both = contract_company_names.isin(identity_company_names)
print(is_name_in_both.value_counts())

contract_company_names[~is_name_in_both]

True     2478
False       5
dtype: int64


1224    FirstLight Power Resources Management, LLC
2367                 Cayuga Operating Company, LLC
2375         Newmont Nevada Energy Investment, LLC
2376                Fortistar North Tonawanda Inc.
2467               VESI Pomona Energy Storage Inc.
dtype: object

In [125]:
all_contracts.to_parquet("2020_contracts.parquet")

In [127]:
list(filter(lambda x: "date" in x, all_contracts.columns))

['contract_execution_date',
 'commencement_date_of_contract_term',
 'contract_termination_date',
 'actual_termination_date',
 'begin_date',
 'end_date']

In [132]:
all_contracts.end_date.astype(pd.Datetime)

AttributeError: module 'pandas' has no attribute 'Datetime'